In [415]:
# webdriver-manager 설치 
# ! pip install webdriver-manager
# Webdriver import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

# Crawling import
import requests
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime

### DB connect import
import pymysql
import pandas as pd
from tqdm import tqdm


### 웹드라이버 로드

In [268]:
def setup_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
    chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

    # install Chrome webdriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

driver = setup_chrome_driver()

### 에펨


In [175]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

# Set up Chrome webdriver
driver = webdriver.Chrome(options=chrome_options)

start = time.time()
word = '그램'
#Selenium_driver로 url 접속
url = f'https://www.fmkorea.com/search.php?act=IS&is_keyword={word}'
driver.get(url)
#Selenium_접속하는 데 시간이 걸릴 수 있으므로 1초 기다림
time.sleep(1)

#BeautifulSoup_html을 Parsing 함
soup = BeautifulSoup(driver.page_source, 'html.parser')
search_in_commu = soup.find('ul', class_="searchResult")
search_results = search_in_commu.findAll("li")
for search in search_in_commu: 
    if search == ' ':
        pass
    else:
        title = search.find('a').get_text().split(']')
        title = ' '.join(']'.join(title[1:]).split(' '))
        url = f"https://www.fmkorea.com{search.find('a')['href']}"
        date = search.find('span', class_ = 'time').get_text().split(' ')[0] # 형식 yyyy-mm-dd
        try:
            comment_cnt = int(search.find('em').get_text())
        except AttributeError:   # em 이 아닌 span에 담겼을 때,
            comment_cnt = 0
        print(title, url, comment_cnt, date)
print(f"소요시간 ==>>> {time.time() - start}")

 라이엇 핼퍼관련해서 제일 웃음벨은 그거 아닌가 https://www.fmkorea.com/5540346457 0 2023-03-01
 당신은 3.1절 경인대첩을 기억하십니까?? https://www.fmkorea.com/5540243150 2 2023-03-01
 아출드 mc 힘의길이 받아서 하면 안될라나 https://www.fmkorea.com/5540272544 11 2023-03-01
 롤계의 광우병 선동  캬하하 헬퍼사건 https://www.fmkorea.com/5540236885 83 2023-03-01
 최근에 GTX1080TI->RTX3070로 하고나서 신세계네요 https://www.fmkorea.com/5540220393 12 2023-03-01
 스압?, 정치인 사진 주의) 불타는 트롯맨의 서혜진 PD에 대하여 알아보자.txt https://www.fmkorea.com/5540092024 0 2023-03-01
 가성비 게이밍 노트북 추천 좀 해주세요 https://www.fmkorea.com/5540177326 0 2023-03-01
 미국에서 화제인 리얼리티 쇼 - MILF MANOR https://www.fmkorea.com/5540052149 5 2023-03-01
 [쿠팡] 레노버 YOGA 6 13ALC R5 Epic Edition (649,000원) (로켓와우무료) https://www.fmkorea.com/5540080517 98 2023-03-01
 PC  본체 견적 문의 겸  부품 추천 문의 https://www.fmkorea.com/5540104314 0 2023-03-01
소요시간 ==>>> 2.9249470233917236


In [165]:
print(search_in_commu)
print(search_in_commu.find('a'))

<ul class="searchResult" id="board_search_list">
</ul>
None


In [169]:
for search in search_in_commu: 
    if search == ' ':
        pass
    else:
        a_tag = search_in_commu.find('a')
        if a_tag == None:
            print('없음')
            break
        print('====',search,'=====')
        print(search)

없음


In [179]:
def fm_crawling(word_list : list):
    '''
    에펨코리아 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''

    start = time.time()
    for word in word_list:
        url = f'https://www.fmkorea.com/search.php?act=IS&is_keyword={word}'
        driver.get(url)
        #Selenium_접속하는 데 시간이 걸릴 수 있으므로 1초 기다림
        time.sleep(1)

        #BeautifulSoup_html을 Parsing 함
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.find('ul', class_="searchResult")
        search_results = search_in_commu.findAll("li")
        for search in search_in_commu: 
            if search != ' ':
                if search_in_commu.find('a') is not None:
                    title = search.find('a').get_text().split(']')
                    title = ' '.join(']'.join(title[1:]).split(' '))
                    url = f"https://www.fmkorea.com{search.find('a')['href']}"
                    date = search.find('span', class_ = 'time').get_text().split(' ')[0] # 형식 yyyy-mm-dd
                    try:
                        comment_cnt = int(search.find('em').get_text())
                    except AttributeError:   # 댓글 없는 경우
                        comment_cnt = 0
                    print(title, url, comment_cnt, date)
    end = time.time() - start
    return end

In [180]:
word_list = ['맥북', '그램', '뉴진스', '미ㅏㅇ러미낭']

fm_crawling(word_list)


 맥북 m1 당근 깡통 https://www.fmkorea.com/5540161651 1 2023-03-01
 메이플 갑자기 개땡기는데 맥북이라 못하는게 억울하다 https://www.fmkorea.com/5540037029 6 2023-03-01
 루다가 260만원 주고 맥북샀는데 우정잉이 210만원에 그 맥북 중고 예약 ㅋㅋ https://www.fmkorea.com/5539952928 2 2023-03-01
 루다 260만원 맥북 언박싱 시작.jpg https://www.fmkorea.com/5539684098 1 2023-03-01
 아이폰 14프로 쓰면서 갤럭시 북샀네 https://www.fmkorea.com/5539668812 3 2023-03-01
 루다 "내 비상금은 내 돈 아니다" 논란 https://www.fmkorea.com/5539631626 1 2023-03-01
 M2 맥북프로 거의한달차 후기. https://www.fmkorea.com/5539489728 3 2023-03-01
 더블 모니터에서 싱글로 갈려고 합니다. https://www.fmkorea.com/5539380316 4 2023-03-01
 [해결함]맥북유전데 질문좀 https://www.fmkorea.com/5539305714 0 2023-03-01
 맥북 14인치에서 16인치로 넘어가면 돈지랄이겠지 https://www.fmkorea.com/5539189193 1 2023-03-01
 ai 동영상.링크 https://www.fmkorea.com/5540447452 2 2023-03-01
 라이엇 핼퍼관련해서 제일 웃음벨은 그거 아닌가 https://www.fmkorea.com/5540346457 0 2023-03-01
 당신은 3.1절 경인대첩을 기억하십니까?? https://www.fmkorea.com/5540243150 2 2023-03-01
 아출드 mc 힘의길이 받아서 하면 안될라나 https://www.fmkorea.com/5540272544 11 2023

8.234305143356323

### 뽐뿌


In [256]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

# Set up Chrome webdriver
driver = webdriver.Chrome(options=chrome_options)

start = time.time()
word = '니ㅏ펀'
#Selenium_driver로 url 접속
url = f'https://www.ppomppu.co.kr/search_bbs.php?bbs_cate=2&keyword={word}'
driver.get(url)
#Selenium_접속하는 데 시간이 걸릴 수 있으므로 1초 기다림
time.sleep(1)

#BeautifulSoup_html을 Parsing 함
soup = BeautifulSoup(driver.page_source, 'html.parser')
search_in_commu = soup.find_all('div', class_="content")
for search in search_in_commu: 
    comment_cnt = search.find('font', class_ = 'comment-cnt').get_text()
    search.find('a').font.decompose()
    # print(search)
    title = search.find('a').get_text()
    url = f"https://www.ppomppu.co.kr{search.find('a')['href']}"
    date = search.find('p', class_ = 'desc').select_one('span:nth-child(3)').string.split('.')
    date = '-'.join(date) # yyyy-mm-dd 형식 맞춰줌
    print(title, url, date, comment_cnt)
print(f"소요시간 ==>>> {time.time() - start}")

소요시간 ==>>> 7.220313549041748


In [259]:
def pp_crawling(driver:webdriver, word_list : list):
    '''
    뽐뿌 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''
    start = time.time()
    for word in word_list:
        url = f'https://www.ppomppu.co.kr/search_bbs.php?bbs_cate=2&keyword={word}'
        driver.get(url)
        time.sleep(1)

        #BeautifulSoup_html을 Parsing 함
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.find_all('div', class_="content")
        for search in search_in_commu: 
            comment_cnt = int(search.find('font', class_ = 'comment-cnt').get_text())
            search.find('a').font.decompose()
            title = search.find('a').get_text()
            url = f"https://www.ppomppu.co.kr{search.find('a')['href']}"
            date = search.find('p', class_ = 'desc').select_one('span:nth-child(3)').string.split('.')
            date = '-'.join(date) # yyyy-mm-dd 형식 맞춰줌
            print(title, url, date, comment_cnt)
    times = time.time() - start
    return times

In [261]:
word_list = ['맥북', '그램', '뉴진스', '니ㅏ펀']

pp_crawling(word_list)

이젠 아이패드가 싸게 느껴지네요 https://www.ppomppu.co.kr/zboard/view.php?id=tablet&no=147651&keyword=%B8%C6%BA%CF 2023-03-01 0
멀티 충전기 (20,500원/무료) https://www.ppomppu.co.kr/zboard/view.php?id=ppomppu8&no=12734&keyword=%B8%C6%BA%CF 2023-03-01 0
Tookci-67W GaN USB C 고속 충전기 ($15.86/무료) https://www.ppomppu.co.kr/zboard/view.php?id=ppomppu8&no=12726&keyword=%B8%C6%BA%CF 2023-02-28 6
애플, MFI 적용된 USB-C 케이블 폭스콘 양산 시작 https://www.ppomppu.co.kr/zboard/view.php?id=phone&no=3750271&keyword=%B8%C6%BA%CF 2023-02-28 37
다이소 맥세이프 호환용 팝핑거 https://www.ppomppu.co.kr/zboard/view.php?id=receive&no=65037&keyword=%B8%C6%BA%CF 2023-02-28 0
쿠팡 카드할인이 이상해요 ㅡ.ㅡ;; https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=8237544&keyword=%B8%C6%BA%CF 2023-02-28 3
맥북에어 M2 vs 아이패드프로 M1+키보드 https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=8237460&keyword=%B8%C6%BA%CF 2023-02-28 11
여행필수품! 75W 접지형 고속멀티충전기 클레버타키온 G21CA https://www.ppomppu.co.kr/zboard/view.php?id=review2&no=65035&keyword=%B8%C6%BA%CF 2023-02-28 0
5시반 타임세일 프로그

12.037775039672852

### 웃대
---

- 검색 사이트 바로 검색 안됨
- 검색어 [] 웃긴대학 내부 검색은 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능합니다.
- 해당 내용이 많으면 로딩 페이지 넘어감


In [305]:
#Selenium_driver로 url 접속
url = "http://web.humoruniv.com/main.html"
driver.get(url)
#Selenium_접속하는 데 시간이 걸릴 수 있으므로 1초 기다림
time.sleep(1)

# 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능
# if word 


x_path_search = '//*[@id="search_text"]'
searchbox = driver.find_element(By.XPATH, x_path_search)
searchbox.click()
element = driver.find_element(By.NAME, "search_text")
element.send_keys("1가re")
element.submit()

#BeautifulSoup_html을 Parsing 함
soup = BeautifulSoup(driver.page_source, 'html.parser')

search_in_commu = soup.find_all('td', style ="word-break: break-all;")
for search in search_in_commu: 
    title = search.find('a').get_text()
    print(title)
    url = search.find('a')['href']
    print(url)
    date = search.find('font').get_text().split(' ')[0]
    print(date)
    comment_cnt = search.find('font', color = 'green').get_text().split(' ')[1]
    print(comment_cnt)


In [299]:
def hu_crawling(driver:webdriver, word_list : list):
    for word in word_list:
        # 웃대는 search link에서 바로 검색 안됨
        url = "http://web.humoruniv.com/main.html" 
        driver.get(url)
        time.sleep(1)
        # 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능
        # if word 

        x_path_search = '//*[@id="search_text"]'
        searchbox = driver.find_element(By.XPATH, x_path_search)
        searchbox.click()
        element = driver.find_element(By.NAME, "search_text")
        element.send_keys(word)
        element.submit()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.find_all('td', style ="word-break: break-all;")
        for search in search_in_commu: 
            title = search.find('a').get_text()
            url = search.find('a')['href']
            date = search.find('font').get_text().split(' ')[0]
            comment_cnt = search.find('font', color = 'green').get_text().split(' ')[1]
            print(title, url, date, comment_cnt)

In [ ]:
word_list = ['맥북', '그램', '뉴진스', '니ㅏ펀']

hu_crawling(word_list)

In [358]:
import re

def has_more_than_two_korean_or_four_english(word):
    korean_pattern = re.compile('[가-힣]{2,}') # Matches 2 or more Korean characters
    english_pattern = re.compile('[a-zA-Z]{4,}') # Matches 4 or more English characters

    # Remove all special characters from the word
    cleaned_word = re.sub('[^가-힣a-zA-Z0-9]+', '', word)
    # cleaned_word = re.sub('[-=+,#/?:^.@*"※~ㆍ!』‘|(%}{)\[\]`$_><;&\\\…》”“’·]', '', word)

    # Check if the cleaned word has more than 2 Korean characters or 4 English characters
    if korean_pattern.search(cleaned_word) or english_pattern.search(cleaned_word):
        return cleaned_word,True
    else:
        return cleaned_word,False
    
has_more_than_two_korean_or_four_english('韓国語会話나4**!@#$)!#$^#(*%$(&#@$^@#)!_+!+#$%@#_)$(%@$%)!@#$&):"{}?></;][\|]`~')

('as4아주', 7)

In [ ]:
# https://bonita-sy.tistory.com/174  한국어, 일본어, 한자, 영어, 숫자 정규표현식

def count_word_len(word, ):
    korean_pattern = re.compile('[가-힣]{2,}') # Matches 2 or more Korean characters
    english_pattern = re.compile('[a-zA-Z]{4,}') # Matches 4 or more English characters
    # # Remove all special characters from the word
    cleaned_word = re.sub('[^가-힣a-zA-Z0-9ぁ-ゔァ-ヴー々〆〤一-龥]+', '', word)
    word_len = len(bytes(cleaned_word, 'euc-kr'))
    return cleaned_word, word_len
    # Check if the cleaned word has more than 2 Korean characters or 4 English characters
    if korean_pattern.search(cleaned_word) or english_pattern.search(cleaned_word):
        return cleaned_word,True
    else:
        return cleaned_word,False
    
count_word_len('as4아주** !@#$)!#$^#(*%$(&#@$^@#)!_+!+#$%@#_)$(%@$%)!@#$&):"}{?></;][\|]`~')

In [353]:
a = 'as4* 가'
print(len(bytes(a, 'euc-kr')))

7


### 엠팍


In [418]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

# Set up Chrome webdriver
driver = webdriver.Chrome(options=chrome_options)

start = time.time()
word = '뉴진스'
url = f'https://mlbpark.donga.com/mp/b.php?search_select=sct&search_input={word}&x=0&y=0&select=sct&m=search&b=bullpen&query={word}'
driver.get(url)
time.sleep(1)

# BeautifulSoup_html을 Parsing 함
soup = BeautifulSoup(driver.page_source, 'html.parser')
search_in_commu = soup.select('tr')
# print(search_in_commu)
for search in search_in_commu: 
    if search.find('a') is not None:
        title = search.find('a', class_ = 'txt').get_text()
        url = search.find('a', class_ = 'txt')['href']
        date = search.find('span', class_ = 'date').get_text()
        if ':' in date: # yyyy-mm-dd 형식 맞춰줌
            date = datetime.now().date()
        try:
            comment_cnt = search.find('span', class_ = 'replycnt').get_text()
            comment_cnt = int(re.sub('[\]\[]', '', comment_cnt))
        except AttributeError:
            comment_cnt = 0
        print(title, url, date, comment_cnt)
# print(f"소요시간 ==>>> {time.time() - start}")

뉴진스 해린 https://mlbpark.donga.com/mp/b.php?id=202303010078634848&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-01 2
뉴진스 닮은 꼴 정리.JPG https://mlbpark.donga.com/mp/b.php?id=202303010078633192&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-01 3
[뉴진스] 멜론차트 역사상 최초 두달 연속 월간차트 TOP3.jpg https://mlbpark.donga.com/mp/b.php?id=202303010078632367&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-01 1
23년을 기점으로 걸그룹 컨셉에 대대적인 변화가 일어날거 같긴 한데 https://mlbpark.donga.com/mp/b.php?id=202303010078631758&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-01 5
뉴진스 하니 밀라노 패션위크 화보.jpg https://mlbpark.donga.com/mp/b.php?id=202303010078631506&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-01 3
뉴진스 하니는 한국 남자랑 결혼하겠죠? https://mlbpark.donga.com/mp/b.php?id=202303010078630519&p=1&b=bullpen&

In [421]:
print(type(driver))

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [424]:
def mp_crwaling(driver:webdriver, word_list: list):
    '''
    mlbpark 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''
    
    start = time.time()
    for word in word_list:
        word = '뉴진스'
        url = f'https://mlbpark.donga.com/mp/b.php?search_select=sct&search_input={word}&x=0&y=0&select=sct&m=search&b=bullpen&query={word}'
        driver.get(url)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.select('tr')
        for search in search_in_commu: 
            if search.find('a') is not None:
                title = search.find('a', class_ = 'txt').get_text()
                url = search.find('a', class_ = 'txt')['href']
                date = search.find('span', class_ = 'date').get_text()
                if ':' in date: # 당일 작성 내용은 시간으로 표시되어 오늘 날짜로 넣어줌
                    date = datetime.now().date()
                try:
                    comment_cnt = search.find('span', class_ = 'replycnt').get_text()
                    comment_cnt = int(re.sub('[\]\[]', '', comment_cnt))
                except AttributeError:
                    comment_cnt = 0
                print(title, url, date, comment_cnt)
    return title, url, date, comment_cnt

In [ ]:
word_list = ['맥북', '그램', '뉴진스', '니ㅏ펀']

mp_crwaling(driver, word_list)
